<h1> Text Classification using TensorFlow on Cloud ML Engine </h1>

This notebook illustrates:
<ol>
<li> Creating datasets for Machine Learning using BigQuery
<li> Creating a text classification model using the high-level Estimator API 
<li> Training on Cloud ML Engine
<li> Deploying model
<li> Predicting with model
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%datalab project set -p $PROJECT

The idea is to look at the title of a newspaper article and figure out whether the article came from the New York Times or from TechCrunch. There are very sophisticated approaches that we can try, but this is a pretty straightforward ML problem and so let's go with something very simple.

<h2> Data exploration and preprocessing in BigQuery </h2>
<p>
What does the Hacker News dataset look like?

In [74]:
%bq query
SELECT
  url, title, score
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  LENGTH(title) > 10
  AND score > 10
LIMIT 5

url,title,score
http://www.articulateventures.com/thoughts-on-being-an-employer/salary-negotatiations-whats-possible-when-there-is-no-more-mone/,Salary Negotiations: What is Possible When There's no More Money,256
http://mzl.la/1pBoskZ,A new set of Firefox Developer Tools features,256
http://stage.vambenepe.com/archives/1932,The War On RSS,256
http://blog.uber.com/2011/09/13/uberdata-how-prostitution-and-alcohol-make-uber-better/,How prostitution and alcohol make Uber better,256
http://the-paper-trail.org/blog/?page_id=152,Advanced Computer Science Courses,256


Let's do some regular expression parsing in BigQuery to get the source of the newspaper article from the URL. For example, if the url is http://mobile.nytimes.com/...., I want to be left with <i>nytimes</i>. To ensure that the parsing works for all URLs of interest, I'll group by the source to make sure there are no weird names left. This was an iterative process.

In [60]:
query="""
SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  COUNT(title) AS num_articles
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '\\\.nytimes.com$|\\\.techcrunch.com$|\\\.wired.com$')
  AND LENGTH(title) > 10
  AND score > 10
GROUP BY
  source
"""

In [61]:
import google.datalab.bigquery as bq
df = bq.Query(query).execute().result().to_dataframe()
df.head()

,source,num_articles
0,nytimes,5795
1,wired,2339
2,techcrunch,1377


Now that we have good parsing of the URL to get the source, let's put together a dataset of source and titles. This will be our labeled dataset for machine learning.

In [62]:
query="""
SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  title
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '\\\.nytimes.com$|\\\.techcrunch.com$|\\\.wired.com$')
  AND LENGTH(title) > 10
  AND score > 10
"""
df = bq.Query(query + " LIMIT 10").execute().result().to_dataframe()
df.head()

,source,title
0,nytimes,The High Line Opens Its Third and Final Phase
1,wired,The World's most Ingenious Thief
2,techcrunch,Google To Acquire DocVerse; Office War Heats Up
3,wired,"Snow Leopard Update Blocks Intel Atom, Kills H..."
4,wired,Facebook's Human-Powered Assistant May Just Su...


For ML training, we will need to split our dataset into training and evaluation datasets (and perhaps an independent test dataset if we are going to do model or feature selection based on the evaluation dataset).  A simple way to do this is to use the hash of a well-distributed column in our data (See https://www.oreilly.com/learning/repeatable-sampling-of-data-sets-in-bigquery-for-machine-learning).
<p>
So, let's do that and save the results as CSV files.

In [64]:
traindf = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) > 0").execute().result().to_dataframe()
evaldf  = bq.Query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) = 0").execute().result().to_dataframe()
traindf.head()

,source,title
0,wired,The Mystery of the Canadian Whiskey Fungus
1,wired,Signature of Antimatter Detected in Lightning
2,wired,Confirmed: Ice on Mars. News broken by Twitter.
3,wired,Adobe Plays the Porn Card in Flash Campaign Ag...
4,wired,Fat? Sick? Blame your grandparents' bad habits


In [77]:
traindf.to_csv('train.csv', header=False, index=False, encoding='utf-8', sep='\t')
evaldf.to_csv('eval.csv', header=False, index=False, encoding='utf-8', sep='\t')

In [78]:
!head -3 train.csv

wired	Is Free Will An Illusion?
wired	How ���Gamification��� Can Make Your Customer Service Worse
wired	How GitHub Helps You Hack the Government


In [79]:
!head -3 eval.csv

wired	The Mystery of the Canadian Whiskey Fungus
wired	Signature of Antimatter Detected in Lightning 
wired	Confirmed: Ice on Mars.  News broken by Twitter.


In [80]:
%bash
gsutil cp *.csv gs://${BUCKET}/txtcls1/

Copying file://eval.csv [Content-Type=text/csv]...
Copying file://train.csv [Content-Type=text/csv]...
|
Operation completed over 2 objects/532.7 KiB.                                    


<h2> Create TensorFlow model using TensorFlow's Estimator API </h2>
<p>
First, write an input_fn to read the data.

In [24]:
import shutil
import tensorflow as tf
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers
from tensorflow.contrib.learn.python.learn import learn_runner
import tensorflow.contrib.metrics as metrics

The CNN model I am using requires constant length inputs. So, titles that are less than MAX_DOCUMENT_LENGTH will be padded; longer titles will be truncated.

In [25]:
TARGETS = ['nytimes', 'wired', 'techcrunch']
MAX_DOCUMENT_LENGTH = 20
EMBEDDING_SIZE = 20
N_FILTERS = 10
WINDOW_SIZE = 5
FILTER_SHAPE1 = [WINDOW_SIZE, EMBEDDING_SIZE]
FILTER_SHAPE2 = [WINDOW_SIZE, N_FILTERS]
POOLING_WINDOW = 4
POOLING_STRIDE = 2

Get the vocabulary of words in the training dataset and save it on the cloud for training and serving.

In [20]:
def save_vocab(trainfile, txtcolname, outfilename):
  if trainfile.startswith('gs://'):
    import subprocess
    tmpfile = "vocab.csv"
    subprocess.check_call("gsutil cp {} {}".format(filename, tmpfile).split(" "))
    filename = tmpfile
  else:
    filename = trainfile
  import pandas as pd
  df = pd.read_csv(filename, header=None, sep='\t', names=['source', 'title'])
  # the text to be classified
  vocab_processor = tflearn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
  vocab_processor.fit(df[txtcolname])
  n_words = len(vocab_processor.vocabulary_)
  print('{} words in {} being written to {}'.format(n_words, trainfile, outfilename))
  vocab.save(outfilename)
  
save_vocab('train.csv', 'title', 'gs://{}/txtcls1/vocab_words'.format(BUCKET));

13182 words in train.csv being written to gs://cloud-training-demos-ml/txtcls1/vocab_words


In [26]:
CSV_COLUMNS = ['source', 'title']
LABEL_COLUMN = 'source'
DEFAULTS = [['null'], ['null']]
NUM_EPOCHS = 2
def read_dataset(prefix, vocab, batch_size=20):
  # use prefix to create filename
  filename = 'gs://{}/txtcls1/{}*'.format(BUCKET, prefix)
  if prefix == 'train':
    mode = tf.contrib.learn.ModeKeys.TRAIN
  else:
    mode = tf.contrib.learn.ModeKeys.EVAL
   
  # the actual input function passed to TensorFlow
  def _input_fn():
    num_epochs = NUM_EPOCHS if mode == tf.contrib.learn.ModeKeys.TRAIN else 1

    # could be a path to one file or a file pattern.
    input_file_names = tf.train.match_filenames_once(filename)
    filename_queue = tf.train.string_input_producer(
        input_file_names, num_epochs=num_epochs, shuffle=True)
 
    # read CSV
    reader = tf.TextLineReader()
    _, value = reader.read_up_to(filename_queue, num_records=batch_size)
    value_column = tf.expand_dims(value, -1)
    columns = tf.decode_csv(value_column, record_defaults=DEFAULTS, field_delim='\t')
    features = dict(zip(CSV_COLUMNS, columns))
    label = features.pop(LABEL_COLUMN)
    return features, label
  
  return _input_fn

Next, define a 2-layer convolutional neural network (CNN) model (adapted https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/learn/text_classification_cnn.py). This is a custom estimator, so I'm following the structure defined in https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/blogs/timeseries/rnn_cloudmle.ipynb

In [32]:
def cnn_model(features, target, mode):
  """2 layer ConvNet to predict from sequence of words to a class."""     

  # make input features numeric
  text_vocab = tflearn.preprocessing.VocabularyProcessor.restore('gs://{}/txtcls1/vocab'.format(BUCKET))
  n_words = len(text_vocab.vocabulary_)
  word_indexes = text_vocab.transform(features)
  word_vectors = tf.contrib.layers.embed_sequence(
      word_indexes, vocab_size=n_words, embed_dim=EMBEDDING_SIZE, scope='words')
  word_vectors = tf.expand_dims(word_vectors, 3)
  
  # make targets numeric
  target_vocab = tflearn.preprocessing.VocabularyProcessor(1)
  target_vocab.fit(TARGETS)
  n_classes = len(target_vocab.vocabulary_)
  target = tf.one_hot(target_vocab.transform(target), n_classes, 1, 0)
  
  with tf.variable_scope('CNN_Layer1'):
    # Apply Convolution filtering on input sequence.
    conv1 = tf.contrib.layers.convolution2d(
        word_vectors, N_FILTERS, FILTER_SHAPE1, padding='VALID')
    # Add a RELU for non linearity.
    conv1 = tf.nn.relu(conv1)
    # Max pooling across output of Convolution+Relu.
    pool1 = tf.nn.max_pool(
        conv1,
        ksize=[1, POOLING_WINDOW, 1, 1],
        strides=[1, POOLING_STRIDE, 1, 1],
        padding='SAME')
    # Transpose matrix so that n_filters from convolution becomes width.
    pool1 = tf.transpose(pool1, [0, 1, 3, 2])
  with tf.variable_scope('CNN_Layer2'):
    # Second level of convolution filtering.
    conv2 = tf.contrib.layers.convolution2d(
        pool1, N_FILTERS, FILTER_SHAPE2, padding='VALID')
    # Max across each filter to get useful features for classification.
    pool2 = tf.squeeze(tf.reduce_max(conv2, 1), squeeze_dims=[1])

  # Apply regular WX + B and classification.
  logits = tf.contrib.layers.fully_connected(pool2, n_classes, activation_fn=None)
  loss = tf.losses.softmax_cross_entropy(target, logits)

  train_op = tf.contrib.layers.optimize_loss(
      loss,
      tf.contrib.framework.get_global_step(),
      optimizer='Adam',
      learning_rate=0.01)

  predictions_dict = {
      'class': tf.argmax(logits, 1),
      'prob': tf.nn.softmax(logits)
  }

  eval_metric_ops = {
      "accuracy": tf.metrics.mean_per_class_accuracy(target, logits, n_classes)
  }
  
  return tflearn.ModelFnOps(
      mode=mode,
      predictions=predictions_dict,
      loss=loss,
      train_op=train_op)
      #eval_metric_ops=eval_metric_ops)


To predict with the TensorFlow model, we also need a serving input function. We will want the title from our user.

In [33]:
def serving_input_fn():
    feature_placeholders = {
      'title': tf.placeholder(tf.string, [None]),
    }
    features = {
      key: tf.expand_dims(tensor, -1)
      for key, tensor in feature_placeholders.items()
    }
    return tflearn.utils.input_fn_utils.InputFnOps(
      features,
      None,
      feature_placeholders)

Finally, train!

In [34]:
def get_train():
  return read_dataset('train', vocab)

def get_valid():
  return read_dataset('valid', vocab)

def experiment_fn(output_dir):
    # run experiment
    return tflearn.Experiment(
        tflearn.Estimator(model_fn=cnn_model, model_dir=output_dir),
        train_input_fn=get_train(),
        eval_input_fn=get_valid(),
        eval_metrics={
            'acc': tflearn.MetricSpec(
                metric_fn=metrics.streaming_accuracy
            )
        }
    )

shutil.rmtree('outputdir', ignore_errors=True) # start fresh each time
learn_runner.run(experiment_fn, 'outputdir')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8f2d332fd0>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


TypeError: Expected binary or unicode string, got <generator object transform at 0x7f8f2d3cdc80>

Now that we have the TensorFlow code working on a subset of the data (in the code above, I was reading only the 00001-of-x file), we can package the TensorFlow code up as a Python module and train it on Cloud ML Engine.
<p>
<h2> Train on Cloud ML Engine </h2>
<p>
Training on Cloud ML Engine requires:
<ol>
<li> Making the code a Python package
<li> Using gcloud to submit the training code to Cloud ML Engine
</ol>
<p>
The code in model.py is the same as in the above cells. I just moved it to a file so that I could package it up as a module.
(explore the <a href="babyweight/trainer">directory structure</a>).

In [1]:
%bash
grep "^def" babyweight/trainer/model.py

def read_dataset(prefix, batch_size=512):
def get_wide_deep():
def serving_input_fn():
def experiment_fn(output_dir):


After moving the code to a package, make sure it works standalone. (Note the --pattern and --num_epochs lines so that I am not trying to boil the ocean on my laptop). Even then, this takes about <b>10 minutes</b> in which you won't see any output ...

In [ ]:
%bash
echo "bucket=${BUCKET}"
rm -rf babyweight_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/babyweight
python -m trainer.task \
   --bucket=${BUCKET} \
   --output_dir=babyweight_trained \
   --job-dir=./tmp \
  --pattern="00001-of-" --num_epochs=2

Once the code works in standalone mode, you can run it on Cloud ML Engine.  Because this is on the entire dataset, it will take a while. The training run took about <b> 3 hours </b> for me. You can monitor the job from the GCP console in the Cloud Machine Learning Engine section.

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/babyweight/trained_model
JOBNAME=babyweight_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
#gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=$(pwd)/babyweight/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   -- \
   --bucket=${BUCKET} \
   --output_dir=${OUTDIR} \
   --num_epochs=10

Training finished with a RMSE of 1 lb.  Obviously, this is our first model. We could probably add in some features, discretize the mother's age, and do some hyper-parameter tuning to get to a lower RMSE.  I'll leave that to you.  If you create a better model, I'd love to hear about it -- please do write a short blog post about what you did, and tweet it at me -- @lak_gcp.

In [3]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/babyweight/trained_model'.format(BUCKET))

TensorBoard was started successfully with pid 28187. Click here to access it.

28187

In [4]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 28187


<table width="70%">
<tr><td><img src="weights.png"/></td><td><img src="rmse.png" /></tr>
</table>

<h2> Deploy trained model </h2>
<p>
Deploying the trained model to act as a REST web service is a simple gcloud call.

In [6]:
%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/export/Servo/

gs://cloud-training-demos-ml/babyweight/trained_model/export/Servo/
gs://cloud-training-demos-ml/babyweight/trained_model/export/Servo/1492051542987/


In [7]:
%bash
MODEL_NAME="babyweight"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/export/Servo/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

Deleting and deploying babyweight v1 from gs://cloud-training-demos-ml/babyweight/trained_model/export/Servo/1492051542987/ ... this will take a few minutes


Creating version (this might take a few minutes)......
...........................................................................................................................................................................................................................................................................................done.


<h2> Use model to predict </h2>
<p>
Send a JSON request to the endpoint of the service to make it predict a baby's weight ... I am going to try out how well the model would have predicted the weights of our two kids and a couple of variations while we are at it ...

In [11]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

import google.cloud.ml.features as features
from google.cloud.ml import session_bundle

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1beta1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1beta1_discovery.json')

request_data = {'instances':
  [
      {
        'is_male': 'True',
        'mother_age': 26.0,
        'mother_race': 'Asian Indian',
        'plurality': 1.0,
        'gestation_weeks': 39,
        'mother_married': 'True',
        'cigarette_use': 'False',
        'alcohol_use': 'False'
      },
      {
        'is_male': 'False',
        'mother_age': 29.0,
        'mother_race': 'Asian Indian',
        'plurality': 1.0,
        'gestation_weeks': 38,
        'mother_married': 'True',
        'cigarette_use': 'False',
        'alcohol_use': 'False'
      },
      {
        'is_male': 'True',
        'mother_age': 26.0,
        'mother_race': 'White',
        'plurality': 1.0,
        'gestation_weeks': 39,
        'mother_married': 'True',
        'cigarette_use': 'False',
        'alcohol_use': 'False'
      },
      {
        'is_male': 'True',
        'mother_age': 26.0,
        'mother_race': 'White',
        'plurality': 2.0,
        'gestation_weeks': 37,
        'mother_married': 'True',
        'cigarette_use': 'False',
        'alcohol_use': 'False'
      }
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'babyweight', 'v1')
response = api.projects().predict(body=request_data, name=parent).execute()
print "response={0}".format(response)

response={u'predictions': [{u'outputs': 7.265425682067871}, {u'outputs': 6.78857421875}, {u'outputs': 7.771362781524658}, {u'outputs': 6.1525468826293945}]}


According to the model, our son would have clocked in at 7.3 lbs and our daughter at 6.8 lbs.
<p>
The weights are off by about 0.5 lbs. Pretty cool!

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License